<img src="http://openenergy-platform.org/static/OEP_logo_2_no_text.svg" alt="OpenEnergy Platform" height="100" width="100"  align="left"/>

# OpenEnergyPlatform
<br><br>

## Usage of OpenEnergyPlatform API-Dialect (oedialect)
Repository: https://github.com/openego/oedialect <br>
Documentation: http://oep-data-interface.readthedocs.io/en/latest/api/how_to.html

Please report bugs and improvements here: https://github.com/OpenEnergyPlatform/oedialect/issues <br>

In [1]:
__copyright__ = "Reiner Lemoine Institut"
__license__   = "GNU Affero General Public License Version 3 (AGPL-3.0)"
__url__       = "https://github.com/openego/data_processing/blob/master/LICENSE"
__author__    = "jh-RLI, christian-rli"

## Tutorial: Creating sql tables, reading spatial-data, uploading to the oedb

Takeaways:
- How to create a table on the OEP from a oemetadata file
- How to read sptial data (from .gkpg files) in python
- How to upload this data to the OEP using the OEP-API and the oedialect

### Overview

First we set up the environment with all dependencys (see requirements.txt) and provide the credentials to connect to the OEP. Then we setup our sql tables, we will do this using the oemetadata format in v1.4.0. The metadata strings should be reviewed first in order to avoid unsupported datatypes or other inconsistencies inside the string. This can lead to errors in the next steps. We use the script to create sqlalchemy tables that are derived from the oemetadata and then create the tables on the oep using sqlachemy with oedialect. After that you should always check if the tables exist and are created properly. If this looks fine we can proceed to the next step and import our spatial data into a geopandas dataframe in python and then upload the data using the oedialect again. Geopandas provides all i/o functionality to do so. 

In this tutorial we focus on reading spatial-data from .gpkg files. Have fun!

## 0. Setup

In [1]:
import geopandas as gpd
import getpass
import sqlalchemy as sa
from sqlalchemy.orm import sessionmaker
import oedialect

### Connection to OEP

If we want to upload data to the OEP we first need to connect to it, using our OEP user name and token. 
Note: You ca view your token on your OEP profile page after logging in.  

In [ ]:
# White spaces in the username are fine!
user = input('Enter OEP-username:')
token = getpass.getpass('Token:')

Now we'll create an sql-alchemy-engine. The engine is what 'speaks' oedialect to the data base api. We need to tell it where the data base is and pass our credentials.


In [ ]:
# Create Engine:
OEP_URL = 'openenergy-platform.org' #'193.175.187.164' #'oep.iks.cs.ovgu.de'
OED_STRING = f'postgresql+oedialect://{user}:{token}@{OEP_URL}'

engine = sa.create_engine(OED_STRING)
metadata = sa.MetaData(bind=engine)
print(metadata)

## 1. Creating sql tables from oemetadata

The oemetadata format is a json file-format that has to be created for all data to be uploaded to the oep. One advantage is that the data model with the used data types is included. So it is possible to derive sql tables from it.

### Provide an oemetadata file

In order to create the table first we need to tell python where to find our oemetadata file. To do this we place the reviewd file in the folder "upload-metadata" in the current directoy or provide a path to our oemetadata folder. The skript is able to process all files that are located in a folder. 

### Setup a Table

We need to tell the data base what columns and datatypes we are about to upload. In our case we have four columns, two of which are text, one is integer and the last is float.

In [ ]:
table_name = 'example_dialect_table'
schema_name = 'sandbox'

ExampleTable = sa.Table(
    table_name,
    metadata,
    sa.Column('variable', sa.VARCHAR(50)),
    sa.Column('unit', sa.VARCHAR(50)),
    sa.Column('year', sa.INTEGER),
    sa.Column('value', sa.FLOAT(50)),
    schema=schema_name
)

### Create the new Table

Now we tell our engine to connect to the data base and create the defined table within the chosen schema.

In [ ]:
conn = engine.connect()
print('Connection established')
if not engine.dialect.has_table(conn, table_name, schema_name):
    ExampleTable.create()
    print('Created table')
else:
    print('Table already exists')

## 2. Reading spatial-data and 3. uploading it to the oedb

Geopandas offers functionality for spatial data. The read_file() function can read data from several sources 
e.g. .gkpg, .geojson, ... the function is also able to import the data by using an url that provides the data.

FYI see: https://geopandas.org/io.html

### Read the data

In [2]:
example_gdf = gpd.read_file('../data/TemplateData.csv', layer='')

TypeError: expected str, bytes or os.PathLike object, not UnparsedPath

Looking at the first three lines of our dataframe:

In [ ]:
example_df[:3]

### Insert (upload) data into Table
 
Uploading the information from our DataFrame is now done with a single command. Uploading data in this way will always delete the content of the table and refill it with new values every time. If you change 'replace' to 'append', the data entries will be added to the preexisting ones. (Connecting and uploading may take a minute.)

In [ ]:
Session = sessionmaker(bind=engine)
session = Session()
try: 
    example_df.to_sql(table_name, conn, schema_name, if_exists='replace')
    print('Inserted to ' + table_name)
except Exception as e:
    session.rollback()
    raise
    print('Insert incomplete!')
finally:
    session.close()

You can also insert data manually into the table.

In [ ]:
Session = sessionmaker(bind=engine)
session = Session()
try:
    insert_statement = ExampleTable.insert().values(
        [
            dict(variable='fairy dust', unit='t', year=2020, value=200),
            dict(variable='mana', unit='kg', year=1999, value=120),
            dict(variable='the force', unit='l', year=1998, value=1100)
        ]
    )
    session.execute(insert_statement)
    session.commit()
    print('Insert successful!')
except Exception as e:
    session.rollback()
    raise
    print('Insert incomplete!')
finally:
    session.close()

## 4. Retriving data, verify that the tables exist and data is uploaded successfully

### Select from Table

Now  we can query our table to see if the data arrived.

In [ ]:
Session = sessionmaker(bind=engine)
session = Session()
print(session.query(ExampleTable).all())
session.close()

## Storing Query Result in DataFrame
We can write the results of the query back into a DataFrame, where it's easier to handle.

In [ ]:
Session = sessionmaker(bind=engine)
session = Session()
df = pd.DataFrame(session.query(ExampleTable).all())
session.close()
df

In [ ]:
pip show sqlalchemy

In [ ]:
pip show oedialect